# 0. Libraries

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import date
today = date.today()
path = os.path.dirname(os.getcwd())
print(f'📂 Current working directory: {path}')
print(f'💚 Today is {today}')
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'scripts'))
import ss_api_call as ss

📂 Current working directory: /Users/serenekim/Desktop/PhD/meta-wealth_mobility
💚 Today is 2025-08-19


# 1. Descriptives of thd used Dataset

In [16]:
df = pd.read_csv(os.path.join(path,"data_abstracts" , "true_mobility_studies_617_forKGs_cleaned.csv"))
print(len(df), f"articles in the dataset \n", df.columns)

617 articles in the dataset 
 Index(['id', 'title', 'year', 'doi', 'landing_page', 'abstract_inverted_index',
       'language', 'is_oa', 'oa_status', 'oa_link', 'abstract', 'abstract_sm',
       'authors_sm', 'domain', 'sort_gpt_1', 'sort_gpt_2', 'sort_gpt4o_1',
       'sort_gpt4o_2', 'cited_by', 'len_cited_by', 'ref_count',
       'cited_by_count', 'Q1', 'Q1_1', 'Q2', 'Q2_1', 'Q2_2', 'Q3', 'Q4', 'abs',
       'index', 'category_n1', 'measure', 'specified', 'relevant',
       'category_n2', 'measure_1', 'measure_2', 'category_3', 'file', 'id.1',
       'Question1', 'Question2', 'rq_cat', 'RQ', 'data_cat', 'data_type',
       'type', 'category_1', 'category_2', 'author_raw_names',
       'author_raw_affiliations', 'topic_display_names', 'topic_scores',
       'field_display_names', 'subfield_display_names'],
      dtype='object')


In [23]:
df[['language', 'domain', 'category_1', 'category_2', 'rq_cat', 'data_cat', 'type', 'topic_display_names', 'field_display_names', 'subfield_display_names' ]].describe(include='all').T

,count,unique,top,freq
language,617,1,en,617
domain,615,1,Social Sciences,615
category_1,617,8,Regression‐based Measures,353
category_2,246,8,Regression‐based Measures,55
rq_cat,617,9,Intergenerational Wealth Mobility and Inheritance,235
data_cat,617,13,Panel/Longitudinal Surveys,166
type,617,3,empirical,510
topic_display_names,615,300,['Intergenerational and Educational Inequality...,33
field_display_names,615,61,"['Social Sciences', 'Social Sciences', 'Social...",233
subfield_display_names,615,154,"['Sociology and Political Science', 'Sociology...",64


In [70]:
import plotly.graph_objects as go

# Prepare the options for dropdown
x_options = [
    {'label': 'Category 1', 'value': 'category_1'},
    {'label': 'Category 2', 'value': 'category_2'},
    {'label': 'RQ Category', 'value': 'rq_cat'},
    {'label': 'Data Category', 'value': 'data_cat'},
    {'label': 'Type', 'value': 'type'},
    {'label': 'Year', 'value': 'year'},
]

# Create initial figure
fig = go.Figure()

# Add traces for each x option, only show the first by default
for i, opt in enumerate(x_options):
    visible = [True if i == 0 else False]
    value_counts = df[opt['value']].value_counts(dropna=False)
    fig.add_trace(go.Bar(
        x=value_counts.index.astype(str),
        y=value_counts.values,
        name=opt['label'],
        visible=visible[0],
        text=value_counts.values,
        textposition='auto'
    ))

# Create dropdown buttons
dropdown_buttons = [
    dict(
        label=opt['label'],
        method='update',
        args=[
            {'visible': [i == j for j in range(len(x_options))]},
            {'xaxis': {'title': opt['label']}}
        ]
    )
    for i, opt in enumerate(x_options)
]

# Add annotation for total number of articles
total_articles = len(df)
fig.update_layout(
    updatemenus=[
        dict(
            buttons=dropdown_buttons,
            direction="down",
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.3,
            yanchor="top"
        )
    ],
    xaxis_title=x_options[0]['label'],
    yaxis_title="Count",
    font=dict(size=16, family="Times New Roman"),
    title_x=0.5,
    title_y=0.95,
    annotations=[
        dict(
            text=f"Length of DataFrame: {total_articles}",
            x=1,
            y=1.2,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=16)
        )
    ]
)

fig.show()

In [72]:
fig.write_html(f'{path}/results/Paper/img/descriptives_properties.html', auto_open=True)

In [75]:
df_cit = pd.read_csv(f'{path}/results/weights_properties_citations_noTypeYear.csv')
df_cit

,from_type,from_val,to_type,to_val,source,weight
0,Measure,Regression‐based Measures,RqType,Empirical Estimates and Determinants,citation,163
1,Measure,Regression‐based Measures,Measure,Regression‐based Measures,citation,138
2,Measure,Regression‐based Measures,DataType,Linked Administrative Data,citation,138
3,Measure,Regression‐based Measures,Measure,Rank‐based Measures,citation,138
4,Measure,Regression‐based Measures,Measure,Regression‐based Measures,citation,130
...,...,...,...,...,...,...
4191,RqType,"Policy, Institutional, and Geographic Impacts",RqType,Empirical Estimates and Determinants,citation,1
4192,RqType,"Policy, Institutional, and Geographic Impacts",RqType,Measurement and Methodological Advances,citation,1
4193,RqType,"Policy, Institutional, and Geographic Impacts",DataType,University/Institution Data,citation,1
4194,RqType,"Policy, Institutional, and Geographic Impacts",RqType,Empirical Estimates and Determinants,citation,1


In [ ]:
import plotly.express as px

df_cit['norm_weight'] = df_cit['weight'] / df_cit['weight'].max()

# Create a custom colorscale: gray for 0, then Viridis for >0
custom_colorscale = [
    [0.0, 'lightgray'],   # z == 0
    [1e-8, '#440154'],    # start Viridis just above 0
    [0.111, '#482878'],
    [0.222, '#3e4989'],
    [0.333, '#31688e'],
    [0.444, '#26828e'],
    [0.555, '#1f9e89'],
    [0.666, '#35b779'],
    [0.777, '#6ece58'],
    [0.888, '#b5de2b'],
    [1.0, '#fde725']
]

fig = px.density_heatmap(
    df_cit,
    x='from_val',
    y='to_val',
    z='norm_weight',
    color_continuous_scale=custom_colorscale,
    labels={'from_type': 'From', 'to_type': 'To', 'weight': 'Weight'},
    title='Density Heatmap of Citation Weights by Type'
)
fig.show()

In [87]:
fig.write_html(f"{path}/results/Paper/img/citation_heatmap.html", auto_open=True)

In [91]:
custom_colorscale = [
    [0.0, 'lightgray'],   # z == 0
    [1e-8, '#440154'],    # start Viridis just above 0
    [0.111, '#482878'],
    [0.222, '#3e4989'],
    [0.333, '#31688e'],
    [0.444, '#26828e'],
    [0.555, '#1f9e89'],
    [0.666, '#35b779'],
    [0.777, '#6ece58'],
    [0.888, '#b5de2b'],
    [1.0, '#fde725']
]


fig = px.density_heatmap(
    df_cit,
    x='from_val',
    y='to_val',
    # histnorm='probability',
    color_continuous_scale=custom_colorscale,
    labels={'from_type': 'From', 'to_type': 'To'},
    title='Density Heatmap of Citation Occurrence by Type'
)
fig.show()

In [92]:
fig.write_html(f'{path}/results/Paper/img/citation_heatmap_counts.html', auto_open=True)

# 2. Get `intents` and `isInfluential` from Semantic Scholar

## Load the `intents` of the citations in our dataset

In [2]:
df = pd.read_csv(os.path.join(path,"data_abstracts" , "true_mobility_studies_617_forKGs_cleaned.csv"))
cit = pd.read_csv(os.path.join(path, 'results', 'citation_relationships_OpenAlexIDs.csv'))
len(df), len(cit)
cit.head()

,citingPaper,citedPaper
0,https://openalex.org/W4393244952,https://openalex.org/W2564827629
1,https://openalex.org/W3208235548,https://openalex.org/W2564827629
2,https://openalex.org/W4289222991,https://openalex.org/W2564827629
3,https://openalex.org/W3045264130,https://openalex.org/W2564827629
4,https://openalex.org/W4280580662,https://openalex.org/W2564827629


In [3]:
print(len(df[df['id'].isin(cit['citingPaper'])]), len(df[df['id'].isin(cit['citedPaper'])]))
print(cit['citingPaper'].nunique(), cit['citedPaper'].nunique())
print(len(df)-len(df[df['cited_by_count']<1]['id'].unique()))
print(len(pd.unique(cit[['citingPaper', 'citedPaper']].values.ravel('K'))))

426 276
426 276
477
469


In [4]:
cit['citingDOI'] = cit['citingPaper'].apply(lambda x: df[df['id'] == x]['doi'].values[0] if not df[df['id'] == x]['doi'].empty else None)
cit['citedDOI'] = cit['citedPaper'].apply(lambda x: df[df['id'] == x]['doi'].values[0] if not df[df['id'] == x]['doi'].empty else None)
cit['citingTitle'] = cit['citingPaper'].apply(lambda x: df[df['id'] == x]['title'].values[0] if not df[df['id'] == x]['title'].empty else None)
cit['citedTitle'] = cit['citedPaper'].apply(lambda x: df[df['id'] == x]['title'].values[0] if not df[df['id'] == x]['title'].empty else None)
cit.head()

,citingPaper,citedPaper,citingDOI,citedDOI,citingTitle,citedTitle
0,https://openalex.org/W4393244952,https://openalex.org/W2564827629,https://doi.org/10.1257/app.20210137,https://doi.org/10.1126/science.aal4617,Measuring Absolute Income Mobility: Lessons fr...,The fading American dream: Trends in absolute ...
1,https://openalex.org/W3208235548,https://openalex.org/W2564827629,https://doi.org/10.1111/1475-4932.12641,https://doi.org/10.1126/science.aal4617,Are We Richer than Our Parents Were? Absolute ...,The fading American dream: Trends in absolute ...
2,https://openalex.org/W4289222991,https://openalex.org/W2564827629,https://doi.org/10.1038/s41586-022-04996-4,https://doi.org/10.1126/science.aal4617,Social capital I: measurement and associations...,The fading American dream: Trends in absolute ...
3,https://openalex.org/W3045264130,https://openalex.org/W2564827629,https://doi.org/10.1111/coep.12478,https://doi.org/10.1126/science.aal4617,IMPROVING EQUALITY OF OPPORTUNITY: NEW INSIGHT...,The fading American dream: Trends in absolute ...
4,https://openalex.org/W4280580662,https://openalex.org/W2564827629,https://doi.org/10.1146/annurev-economics-0823...,https://doi.org/10.1126/science.aal4617,The Great Gatsby Curve,The fading American dream: Trends in absolute ...


In [5]:
cit.isna().sum()

citingPaper      0
citedPaper       0
citingDOI      155
citedDOI        25
citingTitle      0
citedTitle       0
dtype: int64

In [6]:
citing_json = cit['citingDOI'].dropna().tolist()
cited_json = cit['citedDOI'].dropna().to_list()

In [7]:
citingDOI_to_s2 = ss.dois_to_paper_ids(citing_json)
citedDOI_to_s2 = ss.dois_to_paper_ids(cited_json)

In [8]:
citingDOI_to_s2_df = pd.DataFrame(citingDOI_to_s2.items(), columns=['doi', 'paperId'])
citedDOI_to_s2_df = pd.DataFrame(citedDOI_to_s2.items(), columns=['doi', 'paperId'])
citedDOI_to_s2_df

,doi,paperId
0,https://doi.org/10.1126/science.aal4617,d71a5599772aa934e261834ba0ea62249d775b75
1,https://doi.org/10.1111/1468-0297.00296,b1d4276bb0a8cbb00bdd822d2a8169c03ea9be01
2,https://doi.org/10.1080/01603477.1980.11489214,2e35e736144665bf6e23ecd01a50f49b8464d228
3,https://doi.org/10.1111/j.1467-985x.2008.00575.x,aaebdc8d807dc1bb5bfd69a4c0cfe3547acb34ef
4,https://doi.org/10.2139/ssrn.3648796,a015a1963eace5bcae957c54151aeb1e2cd06793
...,...,...
260,https://doi.org/10.1016/j.jpubeco.2023.104974,6f12ecab151aee117ee46baad6e77c6a5cd52a22
261,https://doi.org/10.1086/728737,4e442345f6a0a5bdbba9bcaad0114ef3899eb504
262,https://doi.org/10.1093/sf/soad145,None
263,https://doi.org/10.1177/08912424241279561,a3e890955120dc13df11cef1caeffe756d9e2785


In [9]:
cit_mg = cit.merge(citingDOI_to_s2_df, left_on='citingDOI', right_on='doi', how='left', suffixes=('', '_citing'))
cit_mg = cit_mg.merge(citedDOI_to_s2_df, left_on='citedDOI', right_on='doi', how='left', suffixes=('', '_cited'))

In [10]:
cit_mg = cit_mg.drop(columns=['doi', 'doi_cited'])
cit_mg = cit_mg.rename(columns={'paperId': 'citingPaperId', 'paperId_cited': 'citedPaperId'})
cit_mg.head(3)

,citingPaper,citedPaper,citingDOI,citedDOI,citingTitle,citedTitle,citingPaperId,citedPaperId
0,https://openalex.org/W4393244952,https://openalex.org/W2564827629,https://doi.org/10.1257/app.20210137,https://doi.org/10.1126/science.aal4617,Measuring Absolute Income Mobility: Lessons fr...,The fading American dream: Trends in absolute ...,6af595b5ca4a6103a9b9a241a2d51e84284846f4,d71a5599772aa934e261834ba0ea62249d775b75
1,https://openalex.org/W3208235548,https://openalex.org/W2564827629,https://doi.org/10.1111/1475-4932.12641,https://doi.org/10.1126/science.aal4617,Are We Richer than Our Parents Were? Absolute ...,The fading American dream: Trends in absolute ...,None,d71a5599772aa934e261834ba0ea62249d775b75
2,https://openalex.org/W4289222991,https://openalex.org/W2564827629,https://doi.org/10.1038/s41586-022-04996-4,https://doi.org/10.1126/science.aal4617,Social capital I: measurement and associations...,The fading American dream: Trends in absolute ...,None,d71a5599772aa934e261834ba0ea62249d775b75


## Get `paperId` based on titles

In [99]:
# cit_mg.to_csv(os.path.join(path, 'results', 'citation_relationships_OpenAlexIDs_s2ids.csv'), index=False)
cit_mg = pd.read_csv(os.path.join(path, 'results', 'citation_relationships_OpenAlexIDs_s2ids.csv'))

In [100]:
title_citing = cit_mg[cit_mg['citingPaperId'].isna()]
title_cited = cit_mg[cit_mg['citedPaperId'].isna()]
len(title_citing), len(title_cited)

(425, 301)

In [4]:
citingTitle_to_s2 = ss.titles_to_paper_ids(title_citing['citingTitle'].dropna().tolist(), sleep_between=0.2)

r result: 200 {"data": [{"paperId": "07dc9c431568cf0cb529403d1736df8fee714f30", "title": "Are We Richer Than Our Parents Were? Absolute Income Mobility in Australia", "year": 2021, "matchScore": 300.4518}]}

This is rec: {'data': [{'paperId': '07dc9c431568cf0cb529403d1736df8fee714f30', 'title': 'Are We Richer Than Our Parents Were? Absolute Income Mobility in Australia', 'year': 2021, 'matchScore': 300.4518}]}
output: 07dc9c431568cf0cb529403d1736df8fee714f30
r result: 200 {"data": [{"paperId": "9ddf8befca7e62dfa178bbd02f489122cdb29736", "title": "NBER WORKING PAPER SERIES SOCIAL CAPITAL I: MEASUREMENT AND ASSOCIATIONS WITH ECONOMIC MOBILITY", "year": 2022, "matchScore": 153.59067}]}

This is rec: {'data': [{'paperId': '9ddf8befca7e62dfa178bbd02f489122cdb29736', 'title': 'NBER WORKING PAPER SERIES SOCIAL CAPITAL I: MEASUREMENT AND ASSOCIATIONS WITH ECONOMIC MOBILITY', 'year': 2022, 'matchScore': 153.59067}]}
output: 9ddf8befca7e62dfa178bbd02f489122cdb29736
r result: 200 {"data": [{"pape

In [5]:
citingTitle_to_s2_df = pd.DataFrame(citingTitle_to_s2.items(), columns=['title', 'paperId'])
citingTitle_to_s2_df

,title,paperId
0,Are We Richer than Our Parents Were? Absolute ...,07dc9c431568cf0cb529403d1736df8fee714f30
1,Social capital I: measurement and associations...,9ddf8befca7e62dfa178bbd02f489122cdb29736
2,The Great Gatsby Curve,46213a8359dac1c3195af84f92ff9e88f246dd86
3,Intergenerational Income Mobility: Counterfact...,eef527fa8fd30ce296c467247104b48c538a50d6
4,And Yet It Moves: Intergenerational Mobility i...,fe86ef10134f110d242af23ccdeb37402eba612e
...,...,...
68,Intergenerational Earnings Mobility in Japan A...,17a14535cf56feb533c8508db4fa8e880967ca10
69,Absolute Income Mobility: An Empirical Analysis,51219d31ee969c93020cf1b560a154367eb37972
70,Intergenerational economic mobility in New Zea...,2418a75a50ba936927d0a8f2f20b7b07ef7407b6
71,Geographic Effects on Intergenerational Income...,415abec101f902545a2f544052bf062cf817534f


In [10]:
citingTitle_to_s2_df[citingTitle_to_s2_df['paperId'].isna() == True]['title'].values

array(['Understanding the relationship between intergenerational mobility and community violence',
       'BIG DATA IN MACROECONOMICS: NEW INSIGHTS FROM LARGE ADMINISTRATIVE DATASETS Is the United States Still a Land of Opportunity? Recent Trends in Intergenerational Mobility †'],
      dtype=object)

🔍 Checking these two titles-------------------------------------------------------------------------------------------
* "Understanding the relationship..." : Just doesn't exist in Semantic Scholar
* "BIG DATA...": A duplicate of "Is the United States Still a Land of Opportunity?..." But it's a mistake(?) by Semantic Scholar + Not many citations

In [12]:
# cit_mg[cit_mg['citingTitle'] == "Is the United States Still a Land of Opportunity? Recent Trends in Intergenerational Mobility"]
cit_mg[cit_mg['citingTitle'] == "BIG DATA IN MACROECONOMICS: NEW INSIGHTS FROM LARGE ADMINISTRATIVE DATASETS Is the United States Still a Land of Opportunity? Recent Trends in Intergenerational Mobility †"]

,citingPaper,citedPaper,citingDOI,citedDOI,citingTitle,citedTitle,citingPaperId,citedPaperId
1443,https://openalex.org/W2292189473,https://openalex.org/W2122216841,NaN,https://doi.org/10.1257/jep.27.3.79,BIG DATA IN MACROECONOMICS: NEW INSIGHTS FROM ...,"Income Inequality, Equality of Opportunity, an...",NaN,e99f1340c29d37fc33a79568bd7d6529b5c464c1
2218,https://openalex.org/W2292189473,https://openalex.org/W3093752946,NaN,https://doi.org/10.1162/rest.91.4.766,BIG DATA IN MACROECONOMICS: NEW INSIGHTS FROM ...,Trends in Intergenerational Income Mobility,NaN,cecc00f6c54febe77e82b461b7164b688659c2dd


--------------------------------------------------------------------------------------------------------

In [8]:
citedTitle_to_s2 = ss.titles_to_paper_ids(title_cited['citedTitle'].dropna().tolist(), sleep_between=0.2)
citedTitle_to_s2_df = pd.DataFrame(citedTitle_to_s2.items(), columns=['title', 'paperId'])

r result: 200 {"data": [{"paperId": "07dc9c431568cf0cb529403d1736df8fee714f30", "title": "Are We Richer Than Our Parents Were? Absolute Income Mobility in Australia", "year": 2021, "matchScore": 300.45197}]}

This is rec: {'data': [{'paperId': '07dc9c431568cf0cb529403d1736df8fee714f30', 'title': 'Are We Richer Than Our Parents Were? Absolute Income Mobility in Australia', 'year': 2021, 'matchScore': 300.45197}]}
output: 07dc9c431568cf0cb529403d1736df8fee714f30
r result: 200 {"data": [{"paperId": "b07277495dbda8c494dd5330eef29e6888e9046b", "title": "American Exceptionalism in a New Light: A Comparison of Intergenerational Earnings Mobility in the Nordic Countries, the United Kingdom and the United States", "year": 2006, "matchScore": 334.41235}]}

This is rec: {'data': [{'paperId': 'b07277495dbda8c494dd5330eef29e6888e9046b', 'title': 'American Exceptionalism in a New Light: A Comparison of Intergenerational Earnings Mobility in the Nordic Countries, the United Kingdom and the United Sta

In [9]:
citedTitle_to_s2_df

,title,paperId
0,Are We Richer than Our Parents Were? Absolute ...,07dc9c431568cf0cb529403d1736df8fee714f30
1,American Exceptionalism in a New Light: A Comp...,b07277495dbda8c494dd5330eef29e6888e9046b
2,Social capital I: measurement and associations...,9ddf8befca7e62dfa178bbd02f489122cdb29736
3,The Great Gatsby Curve,46213a8359dac1c3195af84f92ff9e88f246dd86
4,Status Traps,ccb9c19c03581a4c3180eb0a50ecae1fafb5886b
5,"Poor dad, poor child? An investigation of inte...",005ac4a73f240cc802d2198f9c2a769d883c3d87
6,Accounting for Intergenerational Income Persis...,bfe3b291f4355bbc930036cdc9870426042fccbc
7,Unequal chances: family background and economi...,8731856a5b4883dd8671d4554cb4a539a9409f83
8,Intergenerational Economic Mobility in the Uni...,7ec99b071f8b72be7b0236cdf8aad7c82f62bebd
9,"Borrowing Constraints, College Aid, and Interg...",79cfa84a45ee0c8cc57cda529c4aaf47212a6835


In [25]:
cit_mg2 = cit_mg.merge(citingTitle_to_s2_df, left_on='citingTitle', right_on='title', how='left')
cit_mg2 = cit_mg2.merge(citedTitle_to_s2_df, left_on='citedTitle', right_on='title', how='left', suffixes=('', '_cited'))
cit_mg2["citingPaperId"] = cit_mg2["citingPaperId"].fillna(cit_mg2["paperId"])
cit_mg2["citedPaperId"] = cit_mg2["citedPaperId"].fillna(cit_mg2["paperId_cited"])
cit_mg2 = cit_mg2.drop(columns=['title', 'title_cited', 'paperId', 'paperId_cited'])
cit_mg2.head(3)

,citingPaper,citedPaper,citingDOI,citedDOI,citingTitle,citedTitle,citingPaperId,citedPaperId
0,https://openalex.org/W4393244952,https://openalex.org/W2564827629,https://doi.org/10.1257/app.20210137,https://doi.org/10.1126/science.aal4617,Measuring Absolute Income Mobility: Lessons fr...,The fading American dream: Trends in absolute ...,6af595b5ca4a6103a9b9a241a2d51e84284846f4,d71a5599772aa934e261834ba0ea62249d775b75
1,https://openalex.org/W3208235548,https://openalex.org/W2564827629,https://doi.org/10.1111/1475-4932.12641,https://doi.org/10.1126/science.aal4617,Are We Richer than Our Parents Were? Absolute ...,The fading American dream: Trends in absolute ...,07dc9c431568cf0cb529403d1736df8fee714f30,d71a5599772aa934e261834ba0ea62249d775b75
2,https://openalex.org/W4289222991,https://openalex.org/W2564827629,https://doi.org/10.1038/s41586-022-04996-4,https://doi.org/10.1126/science.aal4617,Social capital I: measurement and associations...,The fading American dream: Trends in absolute ...,9ddf8befca7e62dfa178bbd02f489122cdb29736,d71a5599772aa934e261834ba0ea62249d775b75


In [33]:
cit_mg2.isna().sum()

citingPaper        0
citedPaper         0
citingDOI        155
citedDOI          25
citingTitle        0
citedTitle         0
citingPaperId      5
citedPaperId       0
dtype: int64

In [ ]:
# cit_mg2.to_csv(os.path.join(path, 'results', 'citation_relationships_S2PaperIDs.csv'), index=False)

## Get `isinfluential` and `intents` based on S2 `PaperId`
Not that we have collected all the paper ids (one may be modified...), we can now collect these two variables.

In [8]:
df, info, ref_map, ref_stats = ss.enrich_with_intents_and_influential(cit_mg2, include_contexts=True, rps=1.0)

hit: cited=d71a5599772aa934e261834ba0ea62249d775b75 citing=6af595b5ca4a6103a9b9a241a2d51e84284846f4
hit: cited=d71a5599772aa934e261834ba0ea62249d775b75 citing=07dc9c431568cf0cb529403d1736df8fee714f30
hit: cited=d71a5599772aa934e261834ba0ea62249d775b75 citing=9ddf8befca7e62dfa178bbd02f489122cdb29736
hit: cited=d71a5599772aa934e261834ba0ea62249d775b75 citing=b8ce4792418f29ede6d94864ca37acf137e30765
hit: cited=d71a5599772aa934e261834ba0ea62249d775b75 citing=14670a7cd2f6e3800590a25019a1052c8ef0ecc2
hit: cited=d71a5599772aa934e261834ba0ea62249d775b75 citing=14dda1e8728e6e51de8c5b3cd99a3ba2f8def36c
hit: cited=d71a5599772aa934e261834ba0ea62249d775b75 citing=47eb9259bb45120eeef4f8be524c4d5068a640f2
hit: cited=d71a5599772aa934e261834ba0ea62249d775b75 citing=755688a913424312bd97a706021a8249b1290591
hit: cited=d71a5599772aa934e261834ba0ea62249d775b75 citing=664e2868a53f0540a5aed11a747783263561808b
hit: cited=d71a5599772aa934e261834ba0ea62249d775b75 citing=eef527fa8fd30ce296c467247104b48c538a50d6


In [13]:
print("Total length:",len(df),"\n" ,df.isna().sum())

Total length: 2586 
 citingPaper                          0
citedPaper                           0
citingDOI                          155
citedDOI                            25
citingTitle                          0
citedTitle                           0
citingPaperId                        5
citedPaperId                         0
intents                              0
edge_isInfluential                 298
contexts                           298
citing_influentialCitationCount    207
cited_influentialCitationCount      33
dtype: int64


In [14]:
info

{'not_found_all': ['0f91d2fd5ef743e1c31ae759c7214b0925123195',
  '10.1002/j.1839-4655.2016.tb01236.x',
  '10.1002/jae.1246',
  '10.1002/jae.682',
  '10.1002/psp.2192',
  '10.1002/soej.12396',
  '10.1002/soej.12611',
  '10.1007/s00148-018-0722-z',
  '10.1007/s00148-021-00851-4',
  '10.1007/s00199-024-01581-w',
  '10.1007/s10834-021-09809-w',
  '10.1007/s10887-021-09201-1',
  '10.1007/s10888-005-9015-3',
  '10.1007/s10888-017-9372-8',
  '10.1007/s10888-021-09483-w',
  '10.1007/s10888-021-09495-6',
  '10.1007/s10888-022-09550-w',
  '10.1007/s10888-023-09563-z',
  '10.1007/s10888-023-09569-7',
  '10.1007/s10888-023-09577-7',
  '10.1007/s10888-023-09585-7',
  '10.1007/s10901-024-10125-6',
  '10.1007/s11111-020-00371-5',
  '10.1007/s11113-018-9477-1',
  '10.1007/s11150-020-09487-x',
  '10.1007/s11205-020-02378-9',
  '10.1007/s11205-020-02579-2',
  '10.1007/s11205-022-03016-2',
  '10.1007/s11205-024-03437-1',
  '10.1007/s11516-008-0002-x',
  '10.1007/s13524-017-0564-4',
  '10.1007/s13524-018-

## Check `error` from `info` in Semantic Scholar web and some of `not_found_all`.

In [35]:
# Errors in Info

"""
{'cited': 'ed57a852be2e2a2cd443ab8a5805c8de9f957d15' 
Not the right paper! Title-search error, 
    --> This should be: "e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6"
{'cited': '0e31b29c30c69b24322953821e19923fa4597911', 
Duplicate in Semantic Scholar but without citation info
    --> This should be: "819ffc3333c9e2188678f76ab2889fdafc436580"
{'cited': 'a1b5b5fe36b059d71671c8d30c9351f40c494c16', 
    --> This is the journal article of preprint: "a4eade1e5fc218befd5a2f02a0ad0255c9f4ff17" 
    No citations are recorded in Semantic Scholar >< Google Scholar
    Don't do any manipulation with this paperId.
"""

errors = ["ed57a852be2e2a2cd443ab8a5805c8de9f957d15", "0e31b29c30c69b24322953821e19923fa4597911", "a1b5b5fe36b059d71671c8d30c9351f40c494c16"]
# print(cit_mg2[cit_mg2['citingPaperId'].isin(errors) | cit_mg2['citedPaperId'].isin(errors)][['citingTitle', 'citedTitle']].values)
# len(cit_mg2[cit_mg2['citingPaperId'].isin(errors) | cit_mg2['citedPaperId'].isin(errors)])

18

In [ ]:
cit_mg2 = cit_mg2.replace({"citingPaperId": {
    'ed57a852be2e2a2cd443ab8a5805c8de9f957d15': "e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6",
    '0e31b29c30c69b24322953821e19923fa4597911': "819ffc3333c9e2188678f76ab2889fdafc436580",},
                           "citedPaperId": {
    'ed57a852be2e2a2cd443ab8a5805c8de9f957d15': "e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6",
    '0e31b29c30c69b24322953821e19923fa4597911': "819ffc3333c9e2188678f76ab2889fdafc436580",},
                                                     })
df = df.replace({"citingPaperId": {
    'ed57a852be2e2a2cd443ab8a5805c8de9f957d15': "e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6",
    '0e31b29c30c69b24322953821e19923fa4597911': "819ffc3333c9e2188678f76ab2889fdafc436580",},
                           "citedPaperId": {
    'ed57a852be2e2a2cd443ab8a5805c8de9f957d15': "e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6",
    '0e31b29c30c69b24322953821e19923fa4597911': "819ffc3333c9e2188678f76ab2889fdafc436580",},
                                                     })

In [45]:
not_found = ['0f91d2fd5ef743e1c31ae759c7214b0925123195',
  '10.1002/j.1839-4655.2016.tb01236.x',
  '10.1002/jae.1246',
  '10.1002/jae.682',
  '10.1002/psp.2192',
  '10.1002/soej.12396',
  '10.1002/soej.12611',
  '10.1007/s00148-018-0722-z',
  '10.1007/s00148-021-00851-4',
  '10.1007/s00199-024-01581-w',
  '10.1007/s10834-021-09809-w',
  '10.1007/s10887-021-09201-1',
  '10.1007/s10888-005-9015-3',
  '10.1007/s10888-017-9372-8',
  '10.1007/s10888-021-09483-w',
  '10.1007/s10888-021-09495-6',
  '10.1007/s10888-022-09550-w',
  '10.1007/s10888-023-09563-z',
  '10.1007/s10888-023-09569-7',
  '10.1007/s10888-023-09577-7',
  '10.1007/s10888-023-09585-7',
  '10.1007/s10901-024-10125-6',
  '10.1007/s11111-020-00371-5',
  '10.1007/s11113-018-9477-1',
  '10.1007/s11150-020-09487-x',
  '10.1007/s11205-020-02378-9',
  '10.1007/s11205-020-02579-2',
  '10.1007/s11205-022-03016-2',
  '10.1007/s11205-024-03437-1',
  '10.1007/s11516-008-0002-x',
  '10.1007/s13524-017-0564-4',
  '10.1007/s13524-018-0693-4',
  '10.1007/s40822-021-00186-1',
  '10.1007/s41996-019-00033-1',
  '10.1016/j.chieco.2014.10.005',
  '10.1016/j.chieco.2015.12.009',
  '10.1016/j.childyouth.2014.03.002',
  '10.1016/j.childyouth.2019.03.017',
  '10.1016/j.econlet.2010.05.018',
  '10.1016/j.econlet.2012.08.039',
  '10.1016/j.econmod.2024.106862',
  '10.1016/j.euroecorev.2022.104359',
  '10.1016/j.iref.2024.103708',
  '10.1016/j.jce.2024.09.001',
  '10.1016/j.jebo.2021.09.005',
  '10.1016/j.jebo.2024.106683',
  '10.1016/j.jhe.2020.101710',
  '10.1016/j.jpolmod.2009.07.004',
  '10.1016/j.jpubeco.2023.104974',
  '10.1016/j.jpubeco.2024.105074',
  '10.1016/j.labeco.2019.06.005',
  '10.1016/j.labeco.2024.102617',
  '10.1016/j.rssm.2020.100479',
  '10.1016/j.rssm.2022.100713',
  '10.1016/j.socscimed.2019.03.042',
  '10.1016/j.ssresearch.2021.102686',
  '10.1016/j.ssresearch.2024.103064',
  '10.1016/j.strueco.2022.10.006',
  '10.1016/s0165-1765(03)00084-3',
  '10.1016/s2095-3119(20)63373-1',
  '10.1017/dem.2018.19',
  '10.1017/s0022050711000040',
  '10.1017/s0022109023001114',
  '10.1017/s004727941300024x',
  '10.1017/s0047279423000442',
  '10.1017/s1365100518000226',
  '10.1017/s1744137414000034',
  '10.1038/s41586-022-04996-4',
  '10.1073/pnas.1810893116',
  '10.1073/pnas.1814688116',
  '10.1073/pnas.1820464116',
  '10.1073/pnas.2107273118',
  '10.1073/pnas.2309066121',
  '10.1073/pnas.2321418121',
  '10.1080/00036840600949421',
  '10.1080/00036846.2022.2041181',
  '10.1080/00220388.2011.646988',
  '10.1080/00220388.2025.2451867',
  '10.1080/00221546.2019.1565882',
  '10.1080/00277738.2016.1262136',
  '10.1080/00779954.2024.2423703',
  '10.1080/01425690801966402',
  '10.1080/01603477.1980.11489214',
  '10.1080/07350015.2016.1189339',
  '10.1080/07350015.2020.1787176',
  '10.1080/09645292.2013.823909',
  '10.1080/1081602x.2014.975149',
  '10.1080/12265080903391842',
  '10.1080/1331677x.2022.2119424',
  '10.1080/13504851.2019.1660759',
  '10.1080/13504851.2023.2226451',
  '10.1080/13504851.2024.2331247',
  '10.1080/1406099x.2021.1993601',
  '10.1080/1540496x.2014.1013853',
  '10.1080/20780389.2019.1574565',
  '10.1086/260831',
  '10.1086/262117',
  '10.1086/661904',
  '10.1086/675501',
  '10.1086/696058',
  '10.1086/696059',
  '10.1086/700888',
  '10.1086/703465',
  '10.1086/712443',
  '10.1086/718980',
  '10.1086/724860',
  '10.1086/725706',
  '10.1086/728737',
  '10.1086/732527',
  '10.1093/cjres/rsad031',
  '10.1093/ej/ueaa062',
  '10.1093/ej/ueab095',
  '10.1093/ej/uez019',
  '10.1093/ereh/heac010',
  '10.1093/esr/jcab054',
  '10.1093/esr/jcz036',
  '10.1093/jae/ejac044',
  '10.1093/jeg/lby026',
  '10.1093/pnasnexus/pgac012',
  '10.1093/pnasnexus/pgae587',
  '10.1093/qje/qjaa005',
  '10.1093/qje/qju022',
  '10.1093/qje/qjy006',
  '10.1093/qje/qjz042',
  '10.1093/restud/rdaa075',
  '10.1093/restud/rdz038',
  '10.1093/sf/soab080',
  '10.1093/sf/soad079',
  '10.1093/sf/soad145',
  '10.1093/sf/sou092',
  '10.1093/sf/sov075',
  '10.1093/sf/sow109',
  '10.1093/sf/sox051',
  '10.1093/sf/sox086',
  '10.1093/sf/soz108',
  '10.1108/ijse-07-2023-0520',
  '10.1108/jes-03-2024-0138',
  '10.1111/1467-8462.12372',
  '10.1111/1467-8462.12461',
  '10.1111/1467-937x.00203',
  '10.1111/1468-0297.00296',
  '10.1111/1468-4446.12374',
  '10.1111/1468-4446.13064',
  '10.1111/1475-4932.12445',
  '10.1111/1475-4932.12641',
  '10.1111/1475-5890.12352',
  '10.1111/asej.12303',
  '10.1111/caje.12178',
  '10.1111/caje.12699',
  '10.1111/cdep.12231',
  '10.1111/coep.12109',
  '10.1111/coep.12478',
  '10.1111/ecge.12072',
  '10.1111/ecin.12185',
  '10.1111/ecin.12899',
  '10.1111/ecoj.12165',
  '10.1111/ecoj.12328',
  '10.1111/ecoj.12525',
  '10.1111/ecoj.12535',
  '10.1111/ecoj.12607',
  '10.1111/ecot.12278',
  '10.1111/ecot.12383',
  '10.1111/ehr.13374',
  '10.1111/fare.12424',
  '10.1111/iere.12602',
  '10.1111/ijsw.12299',
  '10.1111/ijsw.12499',
  '10.1111/j.1465-7295.2010.00320.x',
  '10.1111/j.1467-6419.2011.00690.x',
  '10.1111/j.1467-6435.2009.00436.x',
  '10.1111/j.1467-6435.2011.00505.x',
  '10.1111/j.1467-9442.2005.00415.x',
  '10.1111/j.1467-9485.2005.00355.x',
  '10.1111/j.1467-985x.2008.00575.x',
  '10.1111/j.1467-985x.2012.01053.x',
  '10.1111/j.1468-0297.2006.01105.x',
  '10.1111/j.1468-232x.2007.00458.x',
  '10.1111/j.1468-2397.2012.00887.x',
  '10.1111/j.1468-4446.2010.01310.x',
  '10.1111/j.1475-4991.1990.tb00275.x',
  '10.1111/j.1475-4991.2000.tb00409.x',
  '10.1111/j.1475-4991.2009.00333.x',
  '10.1111/j.1475-4991.2012.00495.x',
  '10.1111/j.1536-7150.2008.00598.x',
  '10.1111/jcpp.13992',
  '10.1111/jors.12731',
  '10.1111/labr.12164',
  '10.1111/manc.12478',
  '10.1111/obes.12146',
  '10.1111/obes.12355',
  '10.1111/rode.12863',
  '10.1111/roiw.12032',
  '10.1111/roiw.12130',
  '10.1111/roiw.12200',
  '10.1111/roiw.12222',
  '10.1111/roiw.12336',
  '10.1111/roiw.12505',
  '10.1111/roiw.12710',
  '10.1111/rsp3.12452',
  '10.1111/sjoe.12069',
  '10.1111/sjoe.12197',
  '10.1111/sjoe.12205',
  '10.1111/sjoe.12219',
  '10.1111/sjoe.12420',
  '10.1111/sjoe.12438',
  '10.1111/spol.12631',
  '10.1111/spol.13073',
  '10.1111/twec.12043',
  '10.1111/twec.12721',
  '10.1111/twec.13100',
  '10.11114/aef.v2i4.1174',
  '10.1126/science.aal4617',
  '10.1142/s0217590823500273',
  '10.1146/annurev-economics-082321-122703',
  '10.1146/annurev-soc-071811-145521',
  '10.1146/annurev-soc-121919-054814',
  '10.1161/jaha.119.015698',
  '10.1162/0034653053970249',
  '10.1162/rest.91.4.766',
  '10.1162/rest_a_01413',
  '10.1163/15691330-12341473',
  '10.1177/0001699316684004',
  '10.1177/0002716214546998',
  '10.1177/000312240507000304',
  '10.1177/0003122414554947',
  '10.1177/0003122416653602',
  '10.1177/0003122419844992',
  '10.1177/0003122419884497',
  '10.1177/0003122420917591',
  '10.1177/00031224231180607',
  '10.1177/00031224241303459',
  '10.1177/00194662231159838',
  '10.1177/0021909619876320',
  '10.1177/0022146518771910',
  '10.1177/0027950108096594',
  '10.1177/00380385221090877',
  '10.1177/0038040720927886',
  '10.1177/0049124116661579',
  '10.1177/0081175019886613',
  '10.1177/0081175019887992',
  '10.1177/0081175020931126',
  '10.1177/01979183221126462',
  '10.1177/01979183231165499',
  '10.1177/0363199008327642',
  '10.1177/0891242417752436',
  '10.1177/08912424241279561',
  '10.1177/10911421221134727',
  '10.1177/152397211601600102',
  '10.1177/15356841231179436',
  '10.1177/1536867x1601600207',
  '10.1177/2378023118805646',
  '10.1177/2378023119831799',
  '10.1177/2378023119881282',
  '10.1177/2378023120951139',
  '10.1177/23780231241276332',
  '10.1186/s12651-020-00272-y',
  '10.1186/s40175-015-0037-3',
  '10.1186/s40176-017-0104-4',
  '10.1257/000282802760015766',
  '10.1257/089533002760278712',
  '10.1257/aer.103.3.168',
  '10.1257/aer.104.5.141',
  '10.1257/app.20200631',
  '10.1257/app.20210137',
  '10.1257/app.20210151',
  '10.1257/app.20220137',
  '10.1257/app.20230403',
  '10.1257/jel.20211413',
  '10.1257/jep.27.3.79',
  '10.1257/mac.20180466',
  '10.1257/mac.20200442',
  '10.1257/pandp.20211113',
  '10.12681/scad.18957',
  '10.13016/jjwi-5o60',
  '10.1353/foc.2006.0012',
  '10.1353/foc.2006.0016',
  '10.1353/jhr.2013.0012',
  '10.1353/lar.0.0089',
  '10.1371/journal.pone.0297444',
  '10.1371/journal.pone.0313710',
  '10.1515/bejeap-2013-0008',
  '10.15195/v3.a3',
  '10.15195/v6.a15',
  '10.15195/v7.a10',
  '10.1542/peds.2021-053509q',
  '10.1590/2238-38752017v717',
  '10.17310/ntj.2013.4.06',
  '10.17485/ijst/2015/v8is7/70422',
  '10.17576/jem-2022-5603-08',
  '10.18374/ijbr-19-1.3',
  '10.18510/hssr.2021.93141',
  '10.20955/es.2023.5',
  '10.2139/ssrn.1003345',
  '10.2139/ssrn.106588',
  '10.2139/ssrn.1066819',
  '10.2139/ssrn.1127128',
  '10.2139/ssrn.116908',
  '10.2139/ssrn.1298244',
  '10.2139/ssrn.139768',
  '10.2139/ssrn.1556555',
  '10.2139/ssrn.1625788',
  '10.2139/ssrn.1792798',
  '10.2139/ssrn.1796588',
  '10.2139/ssrn.1842094',
  '10.2139/ssrn.1913621',
  '10.2139/ssrn.1966690',
  '10.2139/ssrn.1967466',
  '10.2139/ssrn.1976533',
  '10.2139/ssrn.201028',
  '10.2139/ssrn.2011901',
  '10.2139/ssrn.2189783',
  '10.2139/ssrn.2192762',
  '10.2139/ssrn.2192768',
  '10.2139/ssrn.2225960',
  '10.2139/ssrn.2314809',
  '10.2139/ssrn.2314815',
  '10.2139/ssrn.2320329',
  '10.2139/ssrn.2331887',
  '10.2139/ssrn.2335007',
  '10.2139/ssrn.2370189',
  '10.2139/ssrn.2379562',
  '10.2139/ssrn.2460101',
  '10.2139/ssrn.2496826',
  '10.2139/ssrn.2504100',
  '10.2139/ssrn.2608069',
  '10.2139/ssrn.2612376',
  '10.2139/ssrn.2620727',
  '10.2139/ssrn.2647277',
  '10.2139/ssrn.2666697',
  '10.2139/ssrn.2690728',
  '10.2139/ssrn.2699516',
  '10.2139/ssrn.2740221',
  '10.2139/ssrn.2740395',
  '10.2139/ssrn.2742540',
  '10.2139/ssrn.2796563',
  '10.2139/ssrn.2810458',
  '10.2139/ssrn.2822665',
  '10.2139/ssrn.2846490',
  '10.2139/ssrn.2856359',
  '10.2139/ssrn.2879748',
  '10.2139/ssrn.295559',
  '10.2139/ssrn.3078187',
  '10.2139/ssrn.3092714',
  '10.2139/ssrn.3173259',
  '10.2139/ssrn.3269209',
  '10.2139/ssrn.3269576',
  '10.2139/ssrn.3269831',
  '10.2139/ssrn.3286163',
  '10.2139/ssrn.3289097',
  '10.2139/ssrn.3334749',
  '10.2139/ssrn.3352731',
  '10.2139/ssrn.3390263',
  '10.2139/ssrn.3475800',
  '10.2139/ssrn.3541447',
  '10.2139/ssrn.3648796',
  '10.2139/ssrn.3662560',
  '10.2139/ssrn.3668721',
  '10.2139/ssrn.3698814',
  '10.2139/ssrn.3730453',
  '10.2139/ssrn.3760972',
  '10.2139/ssrn.3772061',
  '10.2139/ssrn.3789289',
  '10.2139/ssrn.3816308',
  '10.2139/ssrn.3858188',
  '10.2139/ssrn.3862378',
  '10.2139/ssrn.3871354',
  '10.2139/ssrn.3876242',
  '10.2139/ssrn.3892624',
  '10.2139/ssrn.3939909',
  '10.2139/ssrn.3974997',
  '10.2139/ssrn.4110960',
  '10.2139/ssrn.4113768',
  '10.2139/ssrn.4180259',
  '10.2139/ssrn.4181511',
  '10.2139/ssrn.418646',
  '10.2139/ssrn.4237631',
  '10.2139/ssrn.4249592',
  '10.2139/ssrn.4702937',
  '10.2139/ssrn.473188',
  '10.2139/ssrn.4754539',
  '10.2139/ssrn.4850631',
  '10.2139/ssrn.5081894',
  '10.2139/ssrn.869435',
  '10.2139/ssrn.878675',
  '10.2139/ssrn.898589',
  '10.2139/ssrn.912720',
  '10.2139/ssrn.920645',
  '10.2139/ssrn.958718',
  '10.2202/1935-1682.1711',
  '10.2202/1935-1682.1713',
  '10.2202/1935-1682.1794',
  '10.2202/1935-1682.2203',
  '10.2202/1935-1682.2339',
  '10.2307/146378',
  '10.2307/2109490',
  '10.2307/2553676',
  '10.2307/2554669',
  '10.2307/3089925',
  '10.2307/591139',
  '10.2478/sbe-2020-0011',
  '10.25071/1874-6322.23682',
  '10.26509/frbc-ec-201606',
  '10.26686/lew.v0i0.1661',
  '10.26686/pq.v7i2.4383',
  '10.26710/jbsee.v4i1.364',
  '10.30541/v25i4pp.609-628',
  '10.3138/cpp.2019-072',
  '10.31737/2221-2264-2022-57-5-6',
  '10.3368/jhr.51.1.239',
  '10.3368/jhr.xl.1.169',
  '10.3390/ijerph191811546',
  '10.3390/socsci11060257',
  '10.3390/socsci7120253',
  '10.3390/su11082341',
  '10.3390/su141912559',
  '10.3848/iif.2012.318.3259',
  '10.3982/qe1849',
  '10.3982/qe69',
  '10.3982/qe771',
  '10.4067/s0718-52862011000100008',
  '10.4067/s0718-88702010000200003',
  '10.4236/jss.2019.75006',
  '10.4236/ojbm.2022.103064',
  '10.4236/ojbm.2022.104092',
  '10.5018/economics-ejournal.ja.2019-7',
  '10.5089/9781484396988.001',
  '10.52813/jei.v11i3.179',
  '10.5860/choice.42-6612',
  '10.5860/choice.50-4551',
  '10.5897/jdae2018.1011',
  '10.7465/jkdi.2014.25.4.715',
  '10.7764/laje.51.2.179',
  '10.7764/laje.51.2.247',
  '14670a7cd2f6e3800590a25019a1052c8ef0ecc2',
  '14dda1e8728e6e51de8c5b3cd99a3ba2f8def36c',
  '31f79aeeb3155f24b6f086baea462bdddebe32e3',
  '3fdc5cd7485b5fc1881333c3a1b701dddd8007e9',
  '4e442345f6a0a5bdbba9bcaad0114ef3899eb504',
  '4f8934ef2563dd9dce3d76a928e1904bf73d5013',
  '52843962a2f1b1d347c3bfe02032ec950ce02953',
  '73028feed3e5fd64ad69c54ff08eb5170f40b3a1',
  '755688a913424312bd97a706021a8249b1290591',
  '7b9619a654bbea64621441e0d296fda3250fcc4c',
  '7fc9c143bb4322bed0ea59fdef59918ca5130980',
  '9686d301b252214a15ebcffd88a14cca292ac6e8',
  'a47968dd277b5ac2427c456c39ed1bb36ac28012',
  'b2e086068a5fd9443c3c3cc9ce54f23b84bf0918',
  'b61c4ca3281932afc23495f564a03f43588196f2',
  'd6353538e265f204314b09b16abab0728bcd2d3e',
  'dbc9c9ce60a023ef6739d70f31e9a97547ab84fb',
  'e7a0d9ed0851762ecc3aeeb58a2b52f297244505']

In [52]:
found_not_found = []
locations = []

for v in not_found:
    mask = cit_mg2.isin([v])
    if mask.any().any():
        found_not_found.append(v)
        locs = [(idx, col) for idx, row in mask.iterrows() for col, present in row.items() if present]
        locations.append({'value': v, 'locations': locs})

locations_df = pd.DataFrame(locations, columns=['value', 'locations'])
locations_df

,value,locations
0,0f91d2fd5ef743e1c31ae759c7214b0925123195,"[(43, citingPaperId), (147, citingPaperId), (3..."
1,14670a7cd2f6e3800590a25019a1052c8ef0ecc2,"[(5, citingPaperId), (247, citedPaperId)]"
2,14dda1e8728e6e51de8c5b3cd99a3ba2f8def36c,"[(6, citingPaperId), (249, citedPaperId), (190..."
3,31f79aeeb3155f24b6f086baea462bdddebe32e3,"[(39, citingPaperId), (81, citingPaperId), (84..."
4,3fdc5cd7485b5fc1881333c3a1b701dddd8007e9,"[(34, citingPaperId), (536, citingPaperId), (5..."
5,4e442345f6a0a5bdbba9bcaad0114ef3899eb504,"[(37, citingPaperId), (179, citingPaperId), (2..."
6,4f8934ef2563dd9dce3d76a928e1904bf73d5013,"[(40, citingPaperId), (729, citingPaperId), (1..."
7,52843962a2f1b1d347c3bfe02032ec950ce02953,"[(20, citingPaperId), (290, citingPaperId), (6..."
8,73028feed3e5fd64ad69c54ff08eb5170f40b3a1,"[(33, citingPaperId), (77, citingPaperId), (31..."
9,755688a913424312bd97a706021a8249b1290591,"[(8, citingPaperId), (271, citingPaperId), (29..."


In [69]:
# Filter the two sets
set_not_found = set(found_not_found)
set_infl_na   = set(df.loc[df["citing_influentialCitationCount"].isna(), "citingPaperId"])
# Intersection
overlap = set_not_found & set_infl_na

print(f"Length of found_not_found: {len(found_not_found)}")
print(f"Overlap count: {len(overlap)}")
print("Sample overlap citingPaperIds:", list(overlap)[:10])

# Optional: see them in a DataFrame
overlap_df = df[df["citingPaperId"].isin(overlap)]
print(len(overlap_df), '\n')
print(overlap_df.isna().sum())

"""
    Okay, these are fine. It's not a problem to consider since that count is not that important.
"""

Length of found_not_found: 19
Overlap count: 19
Sample overlap citingPaperIds: ['9686d301b252214a15ebcffd88a14cca292ac6e8', '31f79aeeb3155f24b6f086baea462bdddebe32e3', 'd6353538e265f204314b09b16abab0728bcd2d3e', '52843962a2f1b1d347c3bfe02032ec950ce02953', '4e442345f6a0a5bdbba9bcaad0114ef3899eb504', '0f91d2fd5ef743e1c31ae759c7214b0925123195', '14670a7cd2f6e3800590a25019a1052c8ef0ecc2', '7fc9c143bb4322bed0ea59fdef59918ca5130980', 'b61c4ca3281932afc23495f564a03f43588196f2', 'dbc9c9ce60a023ef6739d70f31e9a97547ab84fb']
202 

citingPaper                          0
citedPaper                           0
citingDOI                            0
citedDOI                             0
citingTitle                          0
citedTitle                           0
citingPaperId                        0
citedPaperId                         0
intents                              0
edge_isInfluential                  18
contexts                            18
citing_influentialCitationCount    202
cited_

"\n    Okay, these are fine. It's not a problem \n"

In [72]:
# Re-run the code for the two changed paperIds

changed_ids = ["e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6", "819ffc3333c9e2188678f76ab2889fdafc436580"]
error_cit = cit_mg2[cit_mg2['citingPaperId'].isin(changed_ids) | cit_mg2['citedPaperId'].isin(changed_ids)]
error_cit

,citingPaper,citedPaper,citingDOI,citedDOI,citingTitle,citedTitle,citingPaperId,citedPaperId
299,https://openalex.org/W167868874,https://openalex.org/W1557569468,https://doi.org/10.2139/ssrn.2320329,https://doi.org/10.3368/jhr.51.1.239,Intergenerational Earnings Mobility in Korea,Heterogeneous Income Profiles and Lifecycle Bi...,e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6,daf3c3d2b4e8c44cddad924ee167584cf229b664
302,https://openalex.org/W2773177589,https://openalex.org/W1557569468,https://doi.org/10.1186/s40176-017-0104-4,https://doi.org/10.3368/jhr.51.1.239,Intergenerational mobility in Korea,Heterogeneous Income Profiles and Lifecycle Bi...,e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6,daf3c3d2b4e8c44cddad924ee167584cf229b664
385,https://openalex.org/W2208317766,https://openalex.org/W167868874,https://doi.org/10.2139/ssrn.2612376,https://doi.org/10.2139/ssrn.2320329,Intergenerational Mobility of Earnings and Inc...,Intergenerational Earnings Mobility in Korea,b819c97e66eeea6547929eb662f6688960817168,e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6
386,https://openalex.org/W2967279307,https://openalex.org/W167868874,https://doi.org/10.1111/labr.12164,https://doi.org/10.2139/ssrn.2320329,Intergenerational Earnings and Income Mobility...,Intergenerational Earnings Mobility in Korea,a41a2554885f8a20e35d1de1a76b0d89c3a9bfe3,e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6
497,https://openalex.org/W167868874,https://openalex.org/W1989526535,https://doi.org/10.2139/ssrn.2320329,https://doi.org/10.2202/1935-1682.1711,Intergenerational Earnings Mobility in Korea,Comparable Estimates of Intergenerational Inco...,e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6,43082b2d7490dd214d6576ba51133a778080debf
511,https://openalex.org/W2773177589,https://openalex.org/W1989526535,https://doi.org/10.1186/s40176-017-0104-4,https://doi.org/10.2202/1935-1682.1711,Intergenerational mobility in Korea,Comparable Estimates of Intergenerational Inco...,e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6,43082b2d7490dd214d6576ba51133a778080debf
523,https://openalex.org/W167868874,https://openalex.org/W1993865286,https://doi.org/10.2139/ssrn.2320329,https://doi.org/10.1111/roiw.12032,Intergenerational Earnings Mobility in Korea,Intergenerational Mobility in the <scp>U</scp>...,e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6,7f4a80f49d942de888859885e844af6e4008a750
527,https://openalex.org/W2773177589,https://openalex.org/W1993865286,https://doi.org/10.1186/s40176-017-0104-4,https://doi.org/10.1111/roiw.12032,Intergenerational mobility in Korea,Intergenerational Mobility in the <scp>U</scp>...,e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6,7f4a80f49d942de888859885e844af6e4008a750
835,https://openalex.org/W167868874,https://openalex.org/W2053034611,https://doi.org/10.2139/ssrn.2320329,https://doi.org/10.2202/1935-1682.1794,Intergenerational Earnings Mobility in Korea,Intergenerational Earnings Mobility in Italy,e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6,1ec2b2644ca01a5010c603e92e56d076209fad87
853,https://openalex.org/W2773177589,https://openalex.org/W2053034611,https://doi.org/10.1186/s40176-017-0104-4,https://doi.org/10.2202/1935-1682.1794,Intergenerational mobility in Korea,Intergenerational Earnings Mobility in Italy,e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6,1ec2b2644ca01a5010c603e92e56d076209fad87


In [73]:
error_df, error_info, error_ref_map, error_ref_stats = ss.enrich_with_intents_and_influential(error_cit, include_contexts=True)

hit: cited=e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6 citing=b819c97e66eeea6547929eb662f6688960817168
hit: cited=e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6 citing=a41a2554885f8a20e35d1de1a76b0d89c3a9bfe3
hit: cited=43082b2d7490dd214d6576ba51133a778080debf citing=e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6
hit: cited=43082b2d7490dd214d6576ba51133a778080debf citing=e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6
hit: cited=7f4a80f49d942de888859885e844af6e4008a750 citing=e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6
hit: cited=7f4a80f49d942de888859885e844af6e4008a750 citing=e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6
hit: cited=1ec2b2644ca01a5010c603e92e56d076209fad87 citing=e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6
hit: cited=1ec2b2644ca01a5010c603e92e56d076209fad87 citing=e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6
hit: cited=42914c5398bdfe31089f1c735bfe15a07ea1e755 citing=e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6
hit: cited=42914c5398bdfe31089f1c735bfe15a07ea1e755 citing=e8dfecdfbcdc7efaabd855dfa4e698c2d99e8dc6


In [91]:
df_without_errors = df[~(df['citingPaperId'].isin(changed_ids) | df['citedPaperId'].isin(changed_ids))]
df_new = pd.concat([df_without_errors, error_df], ignore_index=False)
len(df_new)

2586

In [94]:
df_new.isna().sum() # 10 less NAs in edge_isInfluential, conext

citingPaper                          0
citedPaper                           0
citingDOI                          155
citedDOI                            25
citingTitle                          0
citedTitle                           0
citingPaperId                        5
citedPaperId                         0
intents                              0
edge_isInfluential                 285
contexts                           285
citing_influentialCitationCount    207
cited_influentialCitationCount      33
dtype: int64

## Extract `citationCount` and `referenceCount` from S2
For the further analysis. + These counts may differ from those from OpenAlex.

In [3]:
cit = pd.read_csv(f"{path}/results/citations_intents_influential_complete.csv") # citations between papers

cited_count_info = ss.fetch_citation_reference_counts(cit['citedPaperId'].unique())
cited_count_info

{'d71a5599772aa934e261834ba0ea62249d775b75': {'citationCount': 758,
  'referenceCount': 72},
 'b1d4276bb0a8cbb00bdd822d2a8169c03ea9be01': {'citationCount': 223,
  'referenceCount': 4},
 '2e35e736144665bf6e23ecd01a50f49b8464d228': {'citationCount': 179,
  'referenceCount': 18},
 'aaebdc8d807dc1bb5bfd69a4c0cfe3547acb34ef': {'citationCount': 31,
  'referenceCount': 13},
 'a015a1963eace5bcae957c54151aeb1e2cd06793': {'citationCount': 14,
  'referenceCount': 71},
 '6af595b5ca4a6103a9b9a241a2d51e84284846f4': {'citationCount': 6,
  'referenceCount': 32},
 '07dc9c431568cf0cb529403d1736df8fee714f30': {'citationCount': 6,
  'referenceCount': 37},
 'b07277495dbda8c494dd5330eef29e6888e9046b': {'citationCount': 483,
  'referenceCount': 50},
 '2bc2fb5b98ed19c6a4778266059b821e7c1497e2': {'citationCount': 220,
  'referenceCount': 77},
 '9ddf8befca7e62dfa178bbd02f489122cdb29736': {'citationCount': 355,
  'referenceCount': 103},
 '426a7214d84865184815857eddd083b926bf3110': {'citationCount': 225,
  'refer

In [4]:
citing_count_info = ss.fetch_citation_reference_counts(cit['citingPaperId'].unique())
citing_count_info

{'6af595b5ca4a6103a9b9a241a2d51e84284846f4': {'citationCount': 6,
  'referenceCount': 32},
 '07dc9c431568cf0cb529403d1736df8fee714f30': {'citationCount': 6,
  'referenceCount': 37},
 '9ddf8befca7e62dfa178bbd02f489122cdb29736': {'citationCount': 355,
  'referenceCount': 103},
 'b8ce4792418f29ede6d94864ca37acf137e30765': {'citationCount': 17,
  'referenceCount': 12},
 '46213a8359dac1c3195af84f92ff9e88f246dd86': {'citationCount': 40,
  'referenceCount': 198},
 '14670a7cd2f6e3800590a25019a1052c8ef0ecc2': {'citationCount': 26,
  'referenceCount': 37},
 '14dda1e8728e6e51de8c5b3cd99a3ba2f8def36c': {'citationCount': 24,
  'referenceCount': 85},
 '47eb9259bb45120eeef4f8be524c4d5068a640f2': {'citationCount': 0,
  'referenceCount': 45},
 '755688a913424312bd97a706021a8249b1290591': {'citationCount': 1,
  'referenceCount': 30},
 '664e2868a53f0540a5aed11a747783263561808b': {'citationCount': 6,
  'referenceCount': 54},
 'eef527fa8fd30ce296c467247104b48c538a50d6': {'citationCount': 10,
  'referenceCou

In [19]:
cited_count_info.keys()

dict_keys(['d71a5599772aa934e261834ba0ea62249d775b75', 'b1d4276bb0a8cbb00bdd822d2a8169c03ea9be01', '2e35e736144665bf6e23ecd01a50f49b8464d228', 'aaebdc8d807dc1bb5bfd69a4c0cfe3547acb34ef', 'a015a1963eace5bcae957c54151aeb1e2cd06793', '6af595b5ca4a6103a9b9a241a2d51e84284846f4', '07dc9c431568cf0cb529403d1736df8fee714f30', 'b07277495dbda8c494dd5330eef29e6888e9046b', '2bc2fb5b98ed19c6a4778266059b821e7c1497e2', '9ddf8befca7e62dfa178bbd02f489122cdb29736', '426a7214d84865184815857eddd083b926bf3110', '101ed5f0a54bbc4a16209a1f313fd823278a8ffe', '886da9cc2d74f88335cfe2ff32e950a9b6cb1f90', '9a45a398072a8e2123bf6dc521ebdae31961bce3', 'bba98683b6d4de2b201aead49a434154e9dfc82d', 'c057329059f8fcdaa64581733cea1e88d871787d', 'b6f0de1fdc8d3026a1f6f522e0cdd621609d0658', '3e7c8985a841222394dbbfc8bcb9cbb6c5d3198f', 'b8ce4792418f29ede6d94864ca37acf137e30765', '732ea2105cee6d6db2afb742ad6ffa89b2422cad', 'df0afec0be9e67272df27359127d1547e3bd95c5', '76f6b339da733bcfc47f3ddc01a1ae2dddc9622b', '318b970119077b8d72e4

In [28]:
cit['cited_CitationsCount'] = None
cit['cited_ReferencesCount'] = None
cit['citing_CitationsCount'] = None
cit['citing_ReferencesCount'] = None

for i in cit['citedPaperId'].unique():
    info = cited_count_info.get(i)
    if info is None:
        print(f"Missing citedPaperId: {i}")
        cit.loc[cit['citedPaperId'] == i, 'cited_CitationsCount'] = 0
        cit.loc[cit['citedPaperId'] == i, 'cited_ReferencesCount'] = 0
    else:
        cit.loc[cit['citedPaperId'] == i, 'cited_CitationsCount'] = info.get('citationCount', 0)
        cit.loc[cit['citedPaperId'] == i, 'cited_ReferencesCount'] = info.get('referenceCount', 0)

for i in cit['citingPaperId'].unique():
    info = citing_count_info.get(i)
    if info is None:
        print(f"Missing citingPaperId: {i}")
        cit.loc[cit['citingPaperId'] == i, 'citing_CitationsCount'] = 0
        cit.loc[cit['citingPaperId'] == i, 'citing_ReferencesCount'] = 0
    else:
        cit.loc[cit['citingPaperId'] == i, 'citing_CitationsCount'] = info.get('citationCount', 0)
        cit.loc[cit['citingPaperId'] == i, 'citing_ReferencesCount'] = info.get('referenceCount', 0)

Missing citingPaperId: nan


In [30]:
cit.isna().sum()

citingPaper                          0
citedPaper                           0
citingDOI                          155
citedDOI                            25
citingTitle                          0
citedTitle                           0
citingPaperId                        5
citedPaperId                         0
intents                              0
edge_isInfluential                 285
contexts                           285
citing_influentialCitationCount    207
cited_influentialCitationCount      33
cited_CitationsCount                 0
cited_ReferencesCount                0
citing_CitationsCount                5
citing_ReferencesCount               5
dtype: int64

In [ ]:
# cit.to_csv(f"{path}/results/citations_intents_influential_complete_all.csv", index=False)

# 3. Analyzing `influential` and `intents` variables

In [32]:
cit = pd.read_csv(f"{path}/results/citations_intents_influential_complete_all.csv") # citations between papers
df = pd.read_csv(f"{path}/data_abstracts/true_mobility_studies_617_forKGs_cleaned.csv") # individiual paper info

In [33]:
print(cit[cit['edge_isInfluential'].isna()]['citingPaperId'].nunique(), cit[cit['edge_isInfluential'].isna()]['citedPaperId'].nunique())
citing_unique = cit[cit['edge_isInfluential'].isna()]['citingPaperId'].unique()
cited_unique = cit[cit['edge_isInfluential'].isna()]['citedPaperId'].unique()
print(len(set(citing_unique) & set(cited_unique)), len(set(citing_unique) | set(cited_unique)))

121 122
36 208


In [34]:
cit_net = cit[cit['edge_isInfluential'].isna() == False].copy()
len(cit_net)

2301

In [35]:
cit_net.isna().sum()

citingPaper                          0
citedPaper                           0
citingDOI                          141
citedDOI                            25
citingTitle                          0
citedTitle                           0
citingPaperId                        0
citedPaperId                         0
intents                              0
edge_isInfluential                   0
contexts                             0
citing_influentialCitationCount    184
cited_influentialCitationCount       0
cited_CitationsCount                 0
cited_ReferencesCount                0
citing_CitationsCount                0
citing_ReferencesCount               0
dtype: int64

## #influential/total_citations vs. method-wise citation behaviors

In [95]:
citations = cit_net.groupby('citedPaperId').agg({
    'citingPaperId': pd.Series.nunique,
    'edge_isInfluential': lambda x: x.astype(int).sum(),
    'cited_influentialCitationCount': 'first',
    'citing_CitationsCount': 'first',
    'citing_ReferencesCount': 'first',
    'cited_CitationsCount': 'first',
    'cited_ReferencesCount': 'first'
}).reset_index()

In [96]:
citations = citations.rename(columns={'citingPaperId' : 'count_citingPaperId'})

In [97]:
cit_net['edge_isInfluential'].value_counts(), citations['edge_isInfluential'].sum()

(edge_isInfluential
 False    1951
 True      350
 Name: count, dtype: int64,
 350)

In [98]:
citations['infl_ratio_within'] = citations['edge_isInfluential'] / citations['count_citingPaperId']
citations['infl_ratio_all'] = citations['cited_influentialCitationCount'] / citations['cited_CitationsCount']

In [99]:
citations[['infl_ratio_within', 'infl_ratio_all']].describe()

,infl_ratio_within,infl_ratio_all
count,238.000000,238.000000
mean,0.104458,0.069699
std,0.222636,0.068432
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.059412
75%,0.142857,0.111111
max,2.000000,0.444444


In [100]:
citations[citations['infl_ratio_within']>0.5]

,citedPaperId,count_citingPaperId,edge_isInfluential,cited_influentialCitationCount,citing_CitationsCount,citing_ReferencesCount,cited_CitationsCount,cited_ReferencesCount,infl_ratio_within,infl_ratio_all
13,0dc3463c805b3d505c75670edc55fad535796767,1,1,1.0,1.0,104.0,14,33,1.0,0.071429
31,25df9ab8f0fbd7add4912e984520404a3763ae1c,1,1,2.0,1.0,30.0,51,23,1.0,0.039216
81,57b57bdae659d0c80b52ccfd322665fb1cec71b6,1,2,4.0,86.0,71.0,41,41,2.0,0.097561
121,86ae8e8711c7634ee83f48d180f2d4b6664244a7,1,1,1.0,4.0,82.0,17,82,1.0,0.058824
140,98d6686f42e45e461e9ae33307016c1fa43ad915,1,1,4.0,1.0,55.0,24,35,1.0,0.166667
187,c709767ddeadd7bb28bf3b13b5cb2a795be68ab3,2,2,1.0,10.0,102.0,5,29,1.0,0.200000


In [55]:
cit[cit['citedPaperId'] == 'c709767ddeadd7bb28bf3b13b5cb2a795be68ab3']
"""Manual Check:
The citing DOI's:
- https://doi.org/10.1111/obes.12355	
- https://doi.org/10.2139/ssrn.3078187	
These two are actually the same paper, but OpenAlex had a wrong Title for the second one.
Luckily, their citingPaperIds are the same, so we can just keep one of them.
"""

,citingPaper,citedPaper,citingDOI,citedDOI,citingTitle,citedTitle,citingPaperId,citedPaperId,intents,edge_isInfluential,contexts,citing_influentialCitationCount,cited_influentialCitationCount,cited_CitationsCount,cited_ReferencesCount,citing_CitationsCount,citing_ReferencesCount
1765,https://openalex.org/W2773358030,https://openalex.org/W2237492881,https://doi.org/10.1111/obes.12355,https://doi.org/10.2139/ssrn.2666697,Distributional Effects of a Continuous Treatme...,Your American Dream is Not Mine! A New Approac...,eef527fa8fd30ce296c467247104b48c538a50d6,c709767ddeadd7bb28bf3b13b5cb2a795be68ab3,"['methodology', 'background', 'result']",True,['the percentage of children with income below...,0.0,1.0,5,29,10.0,102.0
1766,https://openalex.org/W3125052842,https://openalex.org/W2237492881,https://doi.org/10.2139/ssrn.3078187,https://doi.org/10.2139/ssrn.2666697,Intergenerational Income Mobility: Counterfact...,Your American Dream is Not Mine! A New Approac...,eef527fa8fd30ce296c467247104b48c538a50d6,c709767ddeadd7bb28bf3b13b5cb2a795be68ab3,"['methodology', 'background', 'result']",True,['the percentage of children with income below...,0.0,1.0,5,29,10.0,102.0
1767,https://openalex.org/W3216038618,https://openalex.org/W2237492881,https://doi.org/10.2139/ssrn.3772061,https://doi.org/10.2139/ssrn.2666697,Hard Working Is More Important than Destiny,Your American Dream is Not Mine! A New Approac...,775bebdc7385423bca00921bcf514f3e53cf1fba,c709767ddeadd7bb28bf3b13b5cb2a795be68ab3,[],False,[],0.0,1.0,5,29,0.0,29.0


In [59]:
cit_net['intents'].unique()

array(['[]', "['methodology', 'background']",
       "['result', 'methodology', 'background']",
       "['result', 'background']", "['background']", "['methodology']",
       "['result']", "['background', 'result']",
       "['background', 'methodology']",
       "['result', 'background', 'methodology']",
       "['result', 'methodology']",
       "['methodology', 'background', 'result']",
       "['methodology', 'result', 'background']",
       "['background', 'methodology', 'result']",
       "['methodology', 'result']",
       "['background', 'result', 'methodology']"], dtype=object)

In [ ]:
# If 'intents' is a string, use str.len; if it's a list, use .apply(len)
# How many have values in the 'intents' column that are not just [].
len(cit_net[cit_net['intents'].apply(lambda x: isinstance(x, str) and len(x) > 2 or isinstance(x, list) and len(x) > 2)])

873

In [62]:
df.columns

Index(['id', 'title', 'year', 'doi', 'landing_page', 'abstract_inverted_index',
       'language', 'is_oa', 'oa_status', 'oa_link', 'abstract', 'abstract_sm',
       'authors_sm', 'domain', 'sort_gpt_1', 'sort_gpt_2', 'sort_gpt4o_1',
       'sort_gpt4o_2', 'cited_by', 'len_cited_by', 'ref_count',
       'cited_by_count', 'Q1', 'Q1_1', 'Q2', 'Q2_1', 'Q2_2', 'Q3', 'Q4', 'abs',
       'index', 'category_n1', 'measure', 'specified', 'relevant',
       'category_n2', 'measure_1', 'measure_2', 'category_3', 'file', 'id.1',
       'Question1', 'Question2', 'rq_cat', 'RQ', 'data_cat', 'data_type',
       'type', 'category_1', 'category_2', 'author_raw_names',
       'author_raw_affiliations', 'topic_display_names', 'topic_scores',
       'field_display_names', 'subfield_display_names'],
      dtype='object')

## Merging

In [75]:
cited_df = df[df['id'].isin(cit_net['citedPaper'].unique())].copy()
citing_df = df[df['id'].isin(cit_net['citingPaper'].unique())].copy()
len(cited_df), cit_net['citedPaper'].nunique(), len(citing_df), cit_net['citingPaper'].nunique()

(251, 251, 395, 395)

In [82]:
cit_net_complete = cit_net.merge(cited_df[['id', 'abs', 'category_1', 'category_2', 'rq_cat', 'data_cat']], left_on='citedPaper', right_on='id', suffixes=('', '_cited'))
cit_net_complete = cit_net_complete.merge(citing_df[['id', 'abs', 'category_1', 'category_2', 'rq_cat', 'data_cat']], left_on='citingPaper', right_on='id', suffixes=('', '_citing'))

In [85]:
cit_net_complete = cit_net_complete.drop(columns=['id', 'id_citing'])
cit_net_complete = cit_net_complete.rename(columns = {
    'category_1': 'category_1_cited',
    'category_2': 'category_2_cited',
    'rq_cat': 'rq_cat_cited',
    'data_cat': 'data_cat_cited',
    'abs': 'abs_cited',
})

In [101]:
print(cit_net_complete.columns, "\n", cit_net_complete.isna().sum())

Index(['citingPaper', 'citedPaper', 'citingDOI', 'citedDOI', 'citingTitle',
       'citedTitle', 'citingPaperId', 'citedPaperId', 'intents',
       'edge_isInfluential', 'contexts', 'citing_influentialCitationCount',
       'cited_influentialCitationCount', 'cited_CitationsCount',
       'cited_ReferencesCount', 'citing_CitationsCount',
       'citing_ReferencesCount', 'abs_cited', 'category_1_cited',
       'category_2_cited', 'rq_cat_cited', 'data_cat_cited', 'abs_citing',
       'category_1_citing', 'category_2_citing', 'rq_cat_citing',
       'data_cat_citing'],
      dtype='object') 
 citingPaper                           0
citedPaper                            0
citingDOI                           141
citedDOI                             25
citingTitle                           0
citedTitle                            0
citingPaperId                         0
citedPaperId                          0
intents                               0
edge_isInfluential                    0
con

In [ ]:
# cit_net_complete.to_csv(f"{path}/results/citations_intents_influential_complete_all_with_abstracts_18082025.csv", index=False)

## Within-measure citations vs. Methodology citation intent

In [2]:
cit_net_complete = pd.read_csv(f"{path}/results/citations_intents_influential_complete_all_with_abstracts_18082025.csv")

In [3]:
counts = {
    "background": cit_net_complete["intents"].apply(lambda x: "background" in x).sum(),
    "methodology": cit_net_complete["intents"].apply(lambda x: "methodology" in x).sum(),
    "result": cit_net_complete["intents"].apply(lambda x: "result" in x).sum(),
}

print(counts)

{'background': 781, 'methodology': 243, 'result': 186}


In [4]:
# Within the same measures citing.
within_measure_df = cit_net_complete[
    cit_net_complete["category_1_cited"] == cit_net_complete["category_1_citing"]
]
counts = {
    "background": within_measure_df["intents"].apply(lambda x: "background" in x).sum(),
    "methodology": within_measure_df["intents"].apply(lambda x: "methodology" in x).sum(),
    "result": within_measure_df["intents"].apply(lambda x: "result" in x).sum(),
}
print(counts)


{'background': 403, 'methodology': 131, 'result': 101}


In [194]:
print(within_measure_df['edge_isInfluential'].value_counts(), "\n")
print("--------------------------------------------------------")
inf_intents = within_measure_df[(within_measure_df['edge_isInfluential'] == True)
            & (within_measure_df['intents'].apply(lambda x: len(x) > 2))] # "[]" is len(x) == 2.
print(len(inf_intents), "have intents and have influential edges.")
print("✅ Among the influential citation edges, how many have methodology?")
# Contains "methodology"
print(len(within_measure_df[(within_measure_df['edge_isInfluential'] == True) 
                            & (within_measure_df['intents'].apply(lambda x: "methodology" in x))]), "have methodology in their values")
# Contains "result"
print(len(within_measure_df[(within_measure_df['edge_isInfluential'] == True) 
                            & (within_measure_df['intents'].apply(lambda x: "result" in x))]), "have result in their values")
# Contains "background"
print(len(within_measure_df[(within_measure_df['edge_isInfluential'] == True) 
                            & (within_measure_df['intents'].apply(lambda x: "background" in x))]), "have background in their values")
# Only "result"
print(len(within_measure_df[(within_measure_df['edge_isInfluential'] == True) & 
                     (within_measure_df['intents'] == "['result']")]), "have result as their only intent")
# Only "background"
print(len(within_measure_df[(within_measure_df['edge_isInfluential'] == True) & 
                     (within_measure_df['intents'] == "['background']")]), "have background as their only intent")
# Both "result" and "background"
print(len(within_measure_df[(within_measure_df['edge_isInfluential'] == True) & 
                     ((within_measure_df['intents'] == "['result', 'background']") | (within_measure_df['intents'] == "['background', 'result']"))]), "have both result and background as their intents")
print("--------------------------------------------------------")

edge_isInfluential
False    1004
True      187
Name: count, dtype: int64 

--------------------------------------------------------
176 have intents and have influential edges.
✅ Among the influential citation edges, how many have methodology?
70 have methodology in their values
72 have result in their values
167 have background in their values
7 have result as their only intent
64 have background as their only intent
35 have both result and background as their intents
--------------------------------------------------------


In [190]:
within_measure_df[(within_measure_df['edge_isInfluential'] == True) & (within_measure_df['intents'].apply(lambda x: "methodology" in x))].intents.value_counts()

intents
['methodology', 'background']              28
['background', 'methodology']              12
['result', 'methodology', 'background']     8
['methodology', 'result', 'background']     8
['background', 'methodology', 'result']     6
['methodology', 'background', 'result']     3
['result', 'background', 'methodology']     2
['result', 'methodology']                   1
['methodology', 'result']                   1
['background', 'result', 'methodology']     1
Name: count, dtype: int64

To - do 
* whether there is a different pattern between influential or not 
* only methodology intented papers
* edge names -> be careful (in my KG)

# 4. Network graphs

## Create a long table

In [25]:
df = cit_net_complete.copy() 
df = df.replace({
    'category_1_cited':  {'Others': 'Others_Measure'},
    'category_1_citing': {'Others': 'Others_Measure'},
    'rq_cat_cited':      {'Others': 'Others_RqType'},
    'rq_cat_citing':     {'Others': 'Others_RqType'},
    'data_cat_cited':    {'Others': 'Others_DataType'},
    'data_cat_citing':   {'Others': 'Others_DataType'},
})


citing_cols = ["category_1_citing", "rq_cat_citing", "data_cat_citing"]
cited_cols  = ["category_1_cited",  "rq_cat_cited",  "data_cat_cited"]

# sanity check they exist (skip missing gracefully or raise)
missing_citing = [c for c in citing_cols if c not in df.columns]
missing_cited  = [c for c in cited_cols  if c not in df.columns]
if missing_citing or missing_cited:
    raise KeyError(f"Missing columns. citing:{missing_citing} cited:{missing_cited}")

# --- melt citing side ---
citing_long = df.melt(
    id_vars=['citingPaperId', 'citedPaperId', 'edge_isInfluential', 'intents'],
    value_vars=citing_cols,
    var_name="citing",
    value_name="citing_value"
)
citing_long["citing"] = citing_long["citing"].str.replace(r"_citing$", "", regex=True)
    
# --- melt cited side ---
cited_long = df.melt(
    id_vars=['citingPaperId', 'citedPaperId', 'edge_isInfluential', 'intents'],
    value_vars=cited_cols,
    var_name="cited",
    value_name="cited_value"
)
cited_long["cited"] = cited_long["cited"].str.replace(r"_cited$", "", regex=True)

# --- align by pair + matching category name (e.g., category_1 with category_1) ---
long_cit_net_complete = citing_long.merge(
    cited_long,
    left_on=['citingPaperId', 'citedPaperId', 'edge_isInfluential', 'intents'], # Remove/add 'citing' to match between/within node types
    right_on=['citingPaperId', 'citedPaperId', 'edge_isInfluential', 'intents'], # Remove/add 'cited' to match between/within node types
    how="left"
)[['citingPaperId', 'citedPaperId', "citing", "citing_value", "cited", "cited_value", 'edge_isInfluential', 'intents']]

# (optional) sort / rename to your preferred casing
long_cit_net_complete = long_cit_net_complete.sort_values(['citingPaperId', 'citedPaperId', "citing"])
long_cit_net_complete = long_cit_net_complete.rename(columns={
    'citingPaperId': "citingPaperId",
    'citedPaperId': "citedPaperId"
})


In [26]:
long_cit_net_complete['citingPaperId'].nunique(), long_cit_net_complete['citedPaperId'].nunique(), cit_net_complete['citingPaperId'].nunique(), cit_net_complete['citedPaperId'].nunique()

(375, 238, 375, 238)

In [27]:
len(long_cit_net_complete)

22491

In [28]:
long_cit_net_complete.head(2)

,citingPaperId,citedPaperId,citing,citing_value,cited,cited_value,edge_isInfluential,intents
4908,0237f09e2a8fff0c73c118f38dc753e9d087176a,0d0ac5ae38e09611c1e53bf9f02eab133053881c,category_1,Regression‐based Measures,category_1,Regression‐based Measures,False,[]
4909,0237f09e2a8fff0c73c118f38dc753e9d087176a,0d0ac5ae38e09611c1e53bf9f02eab133053881c,category_1,Regression‐based Measures,rq_cat,Empirical Estimates and Determinants,False,[]


## Create an aggregated table

In [29]:
agg_long = long_cit_net_complete.groupby(['citing', 'citing_value', 'cited', 'cited_value']).agg({
    'citingPaperId': 'count', # Number of rows (citingPaperId, citedPaperId) pairs
    # 'citedPaperId': pd.Series.nunique, # Unique number of citedPaperIds
    'edge_isInfluential': 'sum',
    'intents': lambda x: list(set(x))  # Combine intents into a list
}).reset_index()

agg_long = agg_long.rename(columns={'citingPaperId': 'weight', 'edge_isInfluential': 'count_edge_isInfluential'})
agg_long['infl_ratio'] = agg_long['count_edge_isInfluential'] / agg_long['weight']
agg_long

,citing,citing_value,cited,cited_value,weight,count_edge_isInfluential,intents,infl_ratio
0,category_1,Absolute Mobility Measures,category_1,Absolute Mobility Measures,19,3,"[[], ['result'], ['result', 'background'], ['m...",0.157895
1,category_1,Absolute Mobility Measures,category_1,Decomposition / Structural Approaches,4,0,"[[], ['methodology']]",0.000000
2,category_1,Absolute Mobility Measures,category_1,Others_Measure,3,0,"[[], ['methodology']]",0.000000
3,category_1,Absolute Mobility Measures,category_1,Rank‐based Measures,19,3,"[[], ['background', 'result'], ['methodology']...",0.157895
4,category_1,Absolute Mobility Measures,category_1,Regression‐based Measures,53,9,"[[], ['result'], ['background', 'result'], ['m...",0.169811
...,...,...,...,...,...,...,...,...
562,rq_cat,Theoretical and Structural Models,rq_cat,Empirical Estimates and Determinants,42,10,"[[], ['background', 'result'], ['background', ...",0.238095
563,rq_cat,Theoretical and Structural Models,rq_cat,Intergenerational Wealth Mobility and Inheritance,16,1,"[[], ['background'], ['methodology', 'backgrou...",0.062500
564,rq_cat,Theoretical and Structural Models,rq_cat,Measurement and Methodological Advances,2,0,"[[], ['background']]",0.000000
565,rq_cat,Theoretical and Structural Models,rq_cat,"Policy, Institutional, and Geographic Impacts",3,0,"[[], ['background'], ['methodology', 'backgrou...",0.000000


## Plots

In [30]:
import graph

fig = graph.plot_feature_graph_single(
        agg_long, from_val = 'citing_value', to_val = 'cited_value', weight = 'weight', color_labels = graph.color_labels)
fig.show()

In [33]:
fig_geom, fig_betw, fig_deg = graph.plot_centrality_subgraphs_single(
        agg_long, from_val = 'citing_value', to_val = 'cited_value', weight = 'weight', color_labels = graph.color_labels)
fig_geom.show()
fig_betw.show()
fig_deg.show()

In [38]:
len(long_cit_net_complete_filtered), len(long_cit_net_complete)

(14994, 22491)

In [34]:
# Try without considering those citing the same node types.
long_cit_net_complete_filtered = long_cit_net_complete[long_cit_net_complete['citing'] != long_cit_net_complete['cited']]

agg_long_filtered = long_cit_net_complete_filtered.groupby(['citing', 'citing_value', 'cited', 'cited_value']).agg({
    'citingPaperId': 'count', # Number of rows (citingPaperId, citedPaperId) pairs
    # 'citedPaperId': pd.Series.nunique, # Unique number of citedPaperIds
    'edge_isInfluential': 'sum',
    'intents': lambda x: list(set(x))  # Combine intents into a list
}).reset_index()

agg_long_filtered = agg_long_filtered.rename(columns={'citingPaperId': 'weight', 'edge_isInfluential': 'count_edge_isInfluential'})
agg_long_filtered['infl_ratio'] = agg_long_filtered['count_edge_isInfluential'] / agg_long_filtered['weight']
agg_long_filtered


,citing,citing_value,cited,cited_value,weight,count_edge_isInfluential,intents,infl_ratio
0,category_1,Absolute Mobility Measures,data_cat,Administrative/Registry Data,18,1,"[[], ['background'], ['methodology', 'result',...",0.055556
1,category_1,Absolute Mobility Measures,data_cat,International Panel Data,8,0,"[[], ['background']]",0.000000
2,category_1,Absolute Mobility Measures,data_cat,Linked Administrative Data,37,8,"[[], ['background', 'result'], ['result', 'bac...",0.216216
3,category_1,Absolute Mobility Measures,data_cat,National Survey Data,3,1,"[['methodology'], ['background']]",0.333333
4,category_1,Absolute Mobility Measures,data_cat,No dataset,9,1,"[[], ['methodology'], ['background']]",0.111111
...,...,...,...,...,...,...,...,...
372,rq_cat,Theoretical and Structural Models,data_cat,International Panel Data,6,2,"[[], ['background', 'result'], ['background'],...",0.333333
373,rq_cat,Theoretical and Structural Models,data_cat,Linked Administrative Data,23,6,"[[], ['background'], ['methodology', 'backgrou...",0.260870
374,rq_cat,Theoretical and Structural Models,data_cat,National Survey Data,5,1,"[[], ['background'], ['background', 'methodolo...",0.200000
375,rq_cat,Theoretical and Structural Models,data_cat,No dataset,33,6,"[[], ['background', 'result'], ['methodology']...",0.181818


In [35]:
import graph

fig = graph.plot_feature_graph_single(
        agg_long_filtered, from_val = 'citing_value', to_val = 'cited_value', weight = 'weight', color_labels = graph.color_labels)
fig.show()

In [36]:
fig_geom, fig_betw, fig_deg = graph.plot_centrality_subgraphs_single(
        agg_long_filtered, from_val = 'citing_value', to_val = 'cited_value', weight = 'weight', color_labels = graph.color_labels)
fig_geom.show()
fig_betw.show()
fig_deg.show()